In [3]:
# Cell 1: Imports and Warnings
import numpy as np
import pandas as pd
from scipy.integrate import simps, cumulative_trapezoid
from scipy.constants import G as Const_G # Gravitational constant (SI units)
import os
import warnings
import sys # For flushing output in loops
from sympy import N
from sympy.physics.wigner import wigner_3j
import math
from pathlib import Path
from joblib import Parallel, delayed # For parallelization
import functools # Added for lru_cache

# Suppress RuntimeWarnings
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in true_divide')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='divide by zero encountered in true_divide')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in power')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='divide by zero encountered in power')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in multiply')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in subtract')
warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered in cast') # For float(N(sympy_val))
warnings.filterwarnings('ignore', category=UserWarning, message='Numerical coinsidence while calculating wigner symbol')

In [4]:
# Cell 2: Top-level caches and functions
# --- Top-level manual caches ---
_wigner_3j_cache_dict = {}
_b_factor_cache_dict = {}

# --- Top-level cached Wigner and B-factor functions ---
def _wigner_3j_sym_top_level_manual_cache(j1, j2, j3, m1, m2, m3):
    args = (j1, j2, j3, m1, m2, m3)
    if args in _wigner_3j_cache_dict:
        return _wigner_3j_cache_dict[args]
    
    val = wigner_3j(j1, j2, j3, m1, m2, m3)
    result = 0.0
    if val is not None:
        try:
            result = float(N(val))
        except TypeError:
            result = float(val)
    _wigner_3j_cache_dict[args] = result
    return result

def _calculate_B_factor_top_level_manual_cache(l1_in, s_in, l2_in, N_in, sign_pm):
    args = (l1_in, s_in, l2_in, N_in, sign_pm)
    if args in _b_factor_cache_dict:
        return _b_factor_cache_dict[args]

    if not (abs(l1_in - s_in) <= l2_in <= (l1_in + s_in)):
        _b_factor_cache_dict[args] = 0.0
        return 0.0
    
    term_parity_val = 1.0
    if sign_pm == 1: term_parity_val = 1.0 + (-1)**(l1_in + s_in + l2_in)
    elif sign_pm == -1: term_parity_val = 1.0 - (-1)**(l1_in + s_in + l2_in)
    else: raise ValueError("sign_pm must be +1 or -1")
    
    if abs(term_parity_val) < 1e-9:
        _b_factor_cache_dict[args] = 0.0
        return 0.0
    
    N_in_int = int(N_in)
    try:
        math.factorial(l1_in + N_in_int)
        math.factorial(l2_in + N_in_int)
        if (l1_in - N_in_int) < 0 or (l2_in - N_in_int) < 0 :
            _b_factor_cache_dict[args] = 0.0
            return 0.0
        sqrt_term_num_val = math.factorial(l1_in + N_in_int) * math.factorial(l2_in + N_in_int)
        sqrt_term_den_val = math.factorial(l1_in - N_in_int) * math.factorial(l2_in - N_in_int)
    except ValueError: 
        _b_factor_cache_dict[args] = 0.0
        return 0.0

    if abs(sqrt_term_den_val) < 1e-30: 
        if abs(sqrt_term_num_val) < 1e-30:
            _b_factor_cache_dict[args] = 0.0
            return 0.0
        _b_factor_cache_dict[args] = np.inf
        return np.inf
            
    sqrt_val_num = np.sqrt(sqrt_term_num_val / sqrt_term_den_val)
    w3j_val = _wigner_3j_sym_top_level_manual_cache(l1_in, s_in, l2_in, -N_in_int, 0, N_in_int)
    
    if abs(w3j_val) < 1e-15:
        _b_factor_cache_dict[args] = 0.0
        return 0.0
    
    result = 0.5 * ((-1)**N_in_int) * term_parity_val * sqrt_val_num * w3j_val
    _b_factor_cache_dict[args] = result
    return result

# --- Top-level function for perturbation coefficients ---
def _get_tilde_perturbation_coeff_from_direct_input_top_level(
    perturbations_direct_coeffs_arg, 
    pert_type, s, t_complex_target, r_values_or_scalar
):
    is_scalar_request = isinstance(r_values_or_scalar, (int, float, np.floating))
    def fetch_user_input_coeff(current_pert_type, current_s, current_t_input, current_r_val_or_scalar):
        if current_pert_type == 'delta_d':
            if not isinstance(current_r_val_or_scalar, (int, float, np.floating)):
                raise ValueError("r_values_or_scalar must be a scalar for delta_d in fetch_user_input_coeff")
            interface_radius = float(current_r_val_or_scalar)
            val = perturbations_direct_coeffs_arg.get(current_pert_type, {}).get(\
                (interface_radius, current_s, current_t_input), 0.0 )
            return float(val)
        else: 
            pert_data = perturbations_direct_coeffs_arg.get(current_pert_type, {}).get(\
                (current_s, current_t_input), None )
            if isinstance(current_r_val_or_scalar, (int, float, np.floating)):
                r_arr = np.array([float(current_r_val_or_scalar)], dtype=float)
                is_scalar_r_loc = True
            else:
                r_arr = np.asarray(current_r_val_or_scalar, dtype=float)
                is_scalar_r_loc = False
            val_arr = np.zeros_like(r_arr, dtype=float)
            if pert_data is None: pass
            elif callable(pert_data): val_arr = np.array(pert_data(r_arr), dtype=float)
            elif isinstance(pert_data, (int, float, np.floating)): val_arr = np.full_like(r_arr, float(pert_data), dtype=float)
            elif isinstance(pert_data, dict) and 'value' in pert_data and 'range' in pert_data:
                v_pert = float(pert_data['value']); r_min, r_max = pert_data['range']
                mask = (r_arr >= r_min) & (r_arr <= r_max); val_arr[mask] = v_pert
            elif isinstance(pert_data, np.ndarray) and pert_data.shape == r_arr.shape: val_arr = pert_data.astype(float)
            else: print(f"Warning: Invalid format for volumetric perturbation {current_pert_type} ({current_s},{current_t_input}). Returning zeros.")
            return val_arr[0] if is_scalar_r_loc else val_arr
    
    if t_complex_target == 0:
        m_s0_input = fetch_user_input_coeff(pert_type, s, 0, r_values_or_scalar)
        return np.asarray(m_s0_input * np.sqrt(4*np.pi), dtype=complex) 
    
    tau = abs(t_complex_target)
    m_s_neg_tau_input = fetch_user_input_coeff(pert_type, s, -tau, r_values_or_scalar) 
    m_s_pos_tau_input = fetch_user_input_coeff(pert_type, s, tau, r_values_or_scalar)  

    re_m_tilde_s_tau = m_s_neg_tau_input / np.sqrt(2.0) 
    im_m_tilde_s_tau = -m_s_pos_tau_input / np.sqrt(2.0) 
    
    m_tilde_s_tau_calculated = re_m_tilde_s_tau + 1j * im_m_tilde_s_tau
    
    if t_complex_target > 0: 
        return np.asarray(m_tilde_s_tau_calculated  * np.sqrt(4*np.pi), dtype=complex)
    else: 
        sign_factor = 1 if tau % 2 == 0 else -1  
        m_tilde_s_neg_tau_calculated = sign_factor * np.conjugate(m_tilde_s_tau_calculated  * np.sqrt(4*np.pi))
        return np.asarray(m_tilde_s_neg_tau_calculated, dtype=complex)

# --- Top-level function for kernels ---
def _calculate_kernels_top_level(
    k1_tuple, k2_tuple, # For l1, l2, s1_char, s2_char
    data_k1, data_k2, s_pert, 
    rho_model_tl, g_model_tl, kappa_model_tl, mu_model_tl, r_stable_tl, # model arrays
    G_const_tl # Gravitational constant
):
    s1_char, n1, l1 = k1_tuple 
    s2_char, n2, l2 = k2_tuple 
    kernels = {}

    B0_plus  = _calculate_B_factor_top_level_manual_cache(l1, s_pert, l2, 0, +1)
    B1_plus  = _calculate_B_factor_top_level_manual_cache(l1, s_pert, l2, 1, +1)
    B1_minus = _calculate_B_factor_top_level_manual_cache(l1, s_pert, l2, 1, -1)
    B2_plus  = _calculate_B_factor_top_level_manual_cache(l1, s_pert, l2, 2, +1)
    B2_minus = _calculate_B_factor_top_level_manual_cache(l1, s_pert, l2, 2, -1)
    
    B_l2_l1_s_1_plus  = _calculate_B_factor_top_level_manual_cache(l2, l1, s_pert, 1, +1)
    B_l2_l1_s_1_minus = _calculate_B_factor_top_level_manual_cache(l2, l1, s_pert, 1, -1)

    B_l1_l2_s_1_plus  = _calculate_B_factor_top_level_manual_cache(l1, l2, s_pert, 1, +1)
    B_l1_l2_s_1_minus = _calculate_B_factor_top_level_manual_cache(l1, l2, s_pert, 1, -1)

    if s1_char == 'S' and s2_char == 'S':
        u1,v1dk,du1,f1,x1,p1,dp1 = data_k1['u'],data_k1['v_div_k'],data_k1['dot_u'],data_k1['f_val'],data_k1['x_val'],data_k1['p_val'],data_k1['dot_p']
        u2,v2dk,du2,f2,x2,p2,dp2 = data_k2['u'],data_k2['v_div_k'],data_k2['dot_u'],data_k2['f_val'],data_k2['x_val'],data_k2['p_val'],data_k2['dot_p']
        
        kernels['T_rho_SS'] = u1*u2*B0_plus + v1dk*v2dk*B1_plus 
        kernels['V_kappa_SS'] = (du1+f1)*(du2+f2)*B0_plus
        
        V_mu_SS_term1 = (1./3.)*(2*du1-f1)*(2*du2-f2)*B0_plus
        V_mu_SS_term2 = x1*x2*B1_plus
        V_mu_SS_term3 = np.divide(v1dk*v2dk, r_stable_tl**2, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0))*B2_plus
        kernels['V_mu_SS'] = V_mu_SS_term1 + V_mu_SS_term2 + V_mu_SS_term3

        V_rho_SS_term1 = (u1*dp2 + dp1*u2 - 0.5*g_model_tl * \
                         (np.divide(4*u1*u2,r_stable_tl,out=np.zeros_like(rho_model_tl),where=r_stable_tl!=0) + f1*u2 + u1*f2) + \
                         8*np.pi*G_const_tl*rho_model_tl*u1*u2)*B0_plus
        V_rho_SS_term2 = np.divide((p1*v2dk + v1dk*p2) + 0.5*g_model_tl*(u1*v2dk + v1dk*u2), \
                                 r_stable_tl, out=np.zeros_like(rho_model_tl), where=r_stable_tl!=0)*B1_plus
        kernels['V_rho_SS'] = V_rho_SS_term1 + V_rho_SS_term2
        
        Vd_SS_term_bg = -kappa_model_tl*kernels['V_kappa_SS'] - mu_model_tl*kernels['V_mu_SS'] - rho_model_tl*kernels['V_rho_SS']
        Vd_SS_kappa_deriv = kappa_model_tl*( \
                                      (2*du1*du2 + du1*f2 + f1*du2)*B0_plus \
                                      - np.divide((du1+f1)*v2dk, r_stable_tl, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0)) * B_l2_l1_s_1_plus 
                                      - np.divide(v1dk*(du2+f2), r_stable_tl, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0)) * B_l1_l2_s_1_plus  
                                      )
        Vd_SS_mu_deriv = mu_model_tl*( \
                                    (2./3.)*(4*du1*du2 - du1*f2 - f1*du2)*B0_plus \
                                    + (data_k1['dot_v_div_k']*x2 + x1*data_k2['dot_v_div_k'])*B1_plus \
                                    - (2./3.)*np.divide((2*du1-f1)*v2dk, r_stable_tl, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0)) * B_l2_l1_s_1_plus \
                                    - (2./3.)*np.divide(v1dk*(2*du2-f2), r_stable_tl, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0)) * B_l1_l2_s_1_plus \
                                    )
        kernels['V_d_SS'] = Vd_SS_term_bg + Vd_SS_kappa_deriv + Vd_SS_mu_deriv
    elif s1_char == 'T' and s2_char == 'T':
        w1dk, dw1dk, z1 = data_k1['w_div_k'], data_k1['dot_w_div_k'], data_k1['z_val']
        w2dk, dw2dk, z2 = data_k2['w_div_k'], data_k2['dot_w_div_k'], data_k2['z_val']
        kernels['T_rho_TT'] = w1dk*w2dk*B1_plus 
        kernels['V_kappa_TT'] = np.zeros_like(rho_model_tl)
        V_mu_TT_term1 = z1*z2*B1_plus
        V_mu_TT_term2 = np.divide(w1dk*w2dk, r_stable_tl**2, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0))*B2_plus
        kernels['V_mu_TT'] = V_mu_TT_term1 + V_mu_TT_term2
        kernels['V_rho_TT'] = np.zeros_like(rho_model_tl)
        Vd_TT_term_bg = -mu_model_tl*kernels['V_mu_TT'] 
        Vd_TT_mu_deriv = mu_model_tl*(dw1dk*z2 + z1*dw2dk)*B1_plus 
        kernels['V_d_TT'] = Vd_TT_term_bg + Vd_TT_mu_deriv
    elif s1_char == 'S' and s2_char == 'T': 
        u1,v1dk,du1,f1,x1,p1,dv1dk = data_k1['u'],data_k1['v_div_k'],data_k1['dot_u'],data_k1['f_val'],data_k1['x_val'],data_k1['p_val'],data_k1['dot_v_div_k']
        w2dk, dw2dk, z2 = data_k2['w_div_k'], data_k2['dot_w_div_k'], data_k2['z_val']
        kernels['T_rho_ST'] = -1j*v1dk*w2dk*B1_minus
        kernels['V_kappa_ST'] = np.zeros_like(rho_model_tl)
        V_mu_ST_term1 = -1j*x1*z2*B1_minus
        V_mu_ST_term2 = -1j*np.divide(v1dk*w2dk, r_stable_tl**2, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0))*B2_minus
        kernels['V_mu_ST'] = V_mu_ST_term1 + V_mu_ST_term2
        kernels['V_rho_ST'] = -1j*np.divide(p1*w2dk + 0.5*g_model_tl*u1*w2dk, r_stable_tl, out=np.zeros_like(rho_model_tl), where=r_stable_tl!=0)*B1_minus
        Vd_ST_term_bg = -mu_model_tl*kernels['V_mu_ST'] - rho_model_tl*kernels['V_rho_ST']
        Vd_ST_kappa_deriv = -kappa_model_tl * (1j * np.divide((du1+f1)*w2dk, r_stable_tl, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0)) * B_l2_l1_s_1_minus ) 
        Vd_ST_mu_deriv = mu_model_tl * ( -(dv1dk*z2 + x1*dw2dk)*B1_minus \
                                         - (2./3.) * (1j * np.divide((2*du1-f1)*w2dk, r_stable_tl, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0)) * B_l2_l1_s_1_minus) )
        kernels['V_d_ST'] = Vd_ST_term_bg + Vd_ST_kappa_deriv + Vd_ST_mu_deriv
    elif s1_char == 'T' and s2_char == 'S': 
        w1dk, dw1dk, z1 = data_k1['w_div_k'], data_k1['dot_w_div_k'], data_k1['z_val']
        u2,v2dk,du2,f2,x2,p2,dv2dk = data_k2['u'],data_k2['v_div_k'],data_k2['dot_u'],data_k2['f_val'],data_k2['x_val'],data_k2['p_val'],data_k2['dot_v_div_k']
        kernels['T_rho_TS'] = 1j*w1dk*v2dk*B1_minus
        kernels['V_kappa_TS'] = np.zeros_like(rho_model_tl)
        V_mu_TS_term1 = 1j*z1*x2*B1_minus
        V_mu_TS_term2 = 1j*np.divide(w1dk*v2dk, r_stable_tl**2, out=np.zeros_like(rho_model_tl), where=(r_stable_tl!=0))*B2_minus
        kernels['V_mu_TS'] = V_mu_TS_term1 + V_mu_TS_term2
        kernels['V_rho_TS'] = 1j*np.divide(w1dk*p2 + 0.5*g_model_tl*w1dk*u2, r_stable_tl, out=np.zeros_like(rho_model_tl), where=r_stable_tl!=0)*B1_minus
        Vd_TS_term_bg = -mu_model_tl*kernels['V_mu_TS'] - rho_model_tl*kernels['V_rho_TS']
        Vd_TS_kappa_deriv = kappa_model_tl * (1j * np.divide(w1dk*(du2+f2), r_stable_tl, out=np.zeros_like(rho_model_tl), where=r_stable_tl!=0) * B_l1_l2_s_1_minus )
        Vd_TS_mu_deriv = mu_model_tl * ( (z1*dv2dk + dw1dk*x2)*B1_minus \
                                       + (2./3.) * (1j * np.divide(w1dk*(2*du2-f2), r_stable_tl, out=np.zeros_like(rho_model_tl), where=r_stable_tl!=0) * B_l1_l2_s_1_minus) )
        kernels['V_d_TS'] = Vd_TS_term_bg + Vd_TS_kappa_deriv + Vd_TS_mu_deriv

    for coupling_type_td in ['SS', 'TT', 'ST', 'TS']:
        if f'T_rho_{coupling_type_td}' in kernels:
             kernels[f'T_d_{coupling_type_td}'] = -rho_model_tl * kernels[f'T_rho_{coupling_type_td}']
    return kernels

# --- Main top-level task function for parallel execution ---
def _calculate_k1k2_tilde_block_job_top_level(
    k1_tuple_job, k2_tuple_job,
    data_k1, data_k2, 
    r_model_tl, rho_model_tl, g_model_tl, kappa_model_tl, mu_model_tl, r_stable_tl, 
    G_const_tl,
    perturbations_direct_coeffs_arg,
    s_max_arg,
    l1_job, l2_job, 
    m_values_l1, m_values_l2, matrix_size_l1m, matrix_size_l2m
):
    s1_char_job, _, _ = k1_tuple_job
    s2_char_job, _, _ = k2_tuple_job
    coupling_type_job = f"{s1_char_job}{s2_char_job}"

    V_tilde_block = np.zeros((matrix_size_l1m, matrix_size_l2m), dtype=complex)
    T_tilde_block = np.zeros((matrix_size_l1m, matrix_size_l2m), dtype=complex)

    for s_pert in range(s_max_arg + 1): 
        if not (abs(l1_job - s_pert) <= l2_job <= (l1_job + s_pert)): continue
        
        kernels_r_s = _calculate_kernels_top_level(
            k1_tuple_job, k2_tuple_job, data_k1, data_k2, s_pert,
            rho_model_tl, g_model_tl, kappa_model_tl, mu_model_tl, r_stable_tl, G_const_tl
        )
        #if f'T_rho_{coupling_type_job}' not in kernels_r_s: continue 
        
        relevant_interface_radii_for_s_pert_d = []
        if 'delta_d' in perturbations_direct_coeffs_arg:
            relevant_interface_radii_for_s_pert_d = sorted(list(set(\
                k_pert_key[0] for k_pert_key in perturbations_direct_coeffs_arg['delta_d'].keys()\
                if k_pert_key[1] == s_pert )))
        
        for t_complex in range(-s_pert, s_pert + 1):
            delta_rho_st_tilde_r = _get_tilde_perturbation_coeff_from_direct_input_top_level(
                perturbations_direct_coeffs_arg, 'delta_rho', s_pert, t_complex, r_model_tl)
            is_rho_pert_present = np.any(np.abs(delta_rho_st_tilde_r) > 1e-20)

            delta_mu_st_tilde_r = _get_tilde_perturbation_coeff_from_direct_input_top_level(
                perturbations_direct_coeffs_arg, 'delta_mu', s_pert, t_complex, r_model_tl)
            is_mu_pert_present = np.any(np.abs(delta_mu_st_tilde_r) > 1e-20)

            delta_kappa_st_tilde_r = _get_tilde_perturbation_coeff_from_direct_input_top_level(
                perturbations_direct_coeffs_arg, 'delta_kappa', s_pert, t_complex, r_model_tl)
            is_kappa_pert_present = np.any(np.abs(delta_kappa_st_tilde_r) > 1e-20)
            
            is_d_pert_present = False
            if relevant_interface_radii_for_s_pert_d:
                for d_i_check in relevant_interface_radii_for_s_pert_d:
                    d_pert_val_for_st = _get_tilde_perturbation_coeff_from_direct_input_top_level(
                        perturbations_direct_coeffs_arg, 'delta_d', s_pert, t_complex, d_i_check)
                    if abs(d_pert_val_for_st) > 1e-20: is_d_pert_present = True; break
            if not (is_rho_pert_present or is_d_pert_present or is_mu_pert_present or is_kappa_pert_present): continue

            kernel_T_rho = kernels_r_s[f'T_rho_{coupling_type_job}']
            kernel_T_d = kernels_r_s.get(f'T_d_{coupling_type_job}', np.zeros_like(r_model_tl))
            kernel_V_rho = kernels_r_s[f'V_rho_{coupling_type_job}']
            kernel_V_mu = kernels_r_s[f'V_mu_{coupling_type_job}']
            kernel_V_kappa = kernels_r_s[f'V_kappa_{coupling_type_job}']
            kernel_V_d = kernels_r_s.get(f'V_d_{coupling_type_job}', np.zeros_like(r_model_tl))
            
            integral_T_rho_val = 0.0j
            if is_rho_pert_present: 
                integral_T_rho_val = simps(delta_rho_st_tilde_r * kernel_T_rho * r_model_tl**2, r_model_tl)
            
            boundary_sum_T_d_val = 0.0j
            if is_d_pert_present:
                for d_i_interface in relevant_interface_radii_for_s_pert_d:
                    delta_d_st_tilde_scalar = _get_tilde_perturbation_coeff_from_direct_input_top_level(
                        perturbations_direct_coeffs_arg, 'delta_d', s_pert, t_complex, d_i_interface)
                    
                    if abs(delta_d_st_tilde_scalar) > 1e-20:
                        # --- START of MODIFICATION ---
                        surface_proximity_threshold = 500.0  # 定义星球外表面附近的范围 (单位: m)
                        R_surface = r_model_tl[-1]
                        jump_Td_val = 0.0j  # 初始化跳变值

                        # 检查边界是否在星球外表面附近
                        if abs(d_i_interface - R_surface) < surface_proximity_threshold:
                            # 是表面微扰：跳变值为 0 - 表面值
                            # kernel_T_d 数组的最后一个元素即为星球表面值
                            jump_Td_val = 0.0 - kernel_T_d[-1]
                        else:
                            # 是内部界面微扰：使用原始的上下界面求差逻辑
                            jump_half_width = 500
                            idx_below_arr = np.where(r_model_tl < (d_i_interface - jump_half_width))[0]
                            idx_above_arr = np.where(r_model_tl >= (d_i_interface + jump_half_width))[0]
                            if len(idx_below_arr) > 0 and len(idx_above_arr) > 0:
                                idx_b, idx_a = idx_below_arr[-1], idx_above_arr[0]
                                if idx_b < idx_a and idx_a < len(r_model_tl):
                                    jump_Td_val = kernel_T_d[idx_a] - kernel_T_d[idx_b]
                        
                        # 根据计算出的跳变值累加
                        boundary_sum_T_d_val += d_i_interface**2 * delta_d_st_tilde_scalar * jump_Td_val
            sum_contrib_T_val = integral_T_rho_val + boundary_sum_T_d_val
            
            integral_V_rho_val = 0.0j
            if is_rho_pert_present: 
                integral_V_rho_val = simps(delta_rho_st_tilde_r * kernel_V_rho * r_model_tl**2, r_model_tl)
            integral_V_mu_val = 0.0j
            if is_mu_pert_present: 
                integral_V_mu_val = simps(delta_mu_st_tilde_r * kernel_V_mu * r_model_tl**2, r_model_tl)
            integral_V_kappa_val = 0.0j
            if is_kappa_pert_present: 
                integral_V_kappa_val = simps(delta_kappa_st_tilde_r * kernel_V_kappa * r_model_tl**2, r_model_tl)
            boundary_sum_V_d_val = 0.0j
            if is_d_pert_present:
                for d_i_interface in relevant_interface_radii_for_s_pert_d:
                    delta_d_st_tilde_scalar = _get_tilde_perturbation_coeff_from_direct_input_top_level(
                        perturbations_direct_coeffs_arg, 'delta_d', s_pert, t_complex, d_i_interface)

                    if abs(delta_d_st_tilde_scalar) > 1e-20:
                        # --- START of MODIFICATION ---
                        surface_proximity_threshold = 500.0  # 定义星球外表面附近的范围 (单位: m)
                        R_surface = r_model_tl[-1]
                        jump_Vd_val = 0.0j  # 初始化跳变值
                        
                        # 检查边界是否在星球外表面附近
                        if abs(d_i_interface - R_surface) < surface_proximity_threshold:
                            # 是表面微扰：跳变值为 0 - 表面值
                            # kernel_V_d 数组的最后一个元素即为星球表面值
                            jump_Vd_val = 0.0 - kernel_V_d[-1]
                        else:
                            # 是内部界面微扰：使用原始的上下界面求差逻辑
                            jump_half_width = 50
                            idx_below_arr = np.where(r_model_tl < (d_i_interface - jump_half_width))[0]
                            idx_above_arr = np.where(r_model_tl >= (d_i_interface + jump_half_width))[0]
                            if len(idx_below_arr) > 0 and len(idx_above_arr) > 0:
                                idx_b, idx_a = idx_below_arr[-1], idx_above_arr[0]
                                if idx_b < idx_a and idx_a < len(r_model_tl):
                                    jump_Vd_val = kernel_V_d[idx_a] - kernel_V_d[idx_b]

                        # 根据计算出的跳变值累加
                        boundary_sum_V_d_val += d_i_interface**2 * delta_d_st_tilde_scalar * jump_Vd_val
            sum_contrib_V_val = integral_V_rho_val + integral_V_mu_val + integral_V_kappa_val + boundary_sum_V_d_val
            
            if abs(sum_contrib_T_val) < 1e-20 and abs(sum_contrib_V_val) < 1e-20: continue
            
            common_3j_prefactor = np.sqrt( (2*l1_job+1) * (2*s_pert+1) * (2*l2_job+1) / (4*np.pi) )
            
            for m1_idx_loop, m1_val_orig in enumerate(m_values_l1):
                m1_val_loop = int(m1_val_orig) 
                for m2_idx_loop, m2_val_orig in enumerate(m_values_l2):
                    m2_val_loop = int(m2_val_orig) 
                    
                    w3j_symbol_val = _wigner_3j_sym_top_level_manual_cache(l1_job, s_pert, l2_job, -m1_val_loop, t_complex, m2_val_loop)
                    if abs(w3j_symbol_val) < 1e-15: continue
                    
                    sign_m1_term = (-1)**m1_val_loop 
                    final_angular_factor = sign_m1_term * common_3j_prefactor * w3j_symbol_val
                    
                    T_tilde_block[m1_idx_loop, m2_idx_loop] += final_angular_factor * sum_contrib_T_val
                    V_tilde_block[m1_idx_loop, m2_idx_loop] += final_angular_factor * sum_contrib_V_val
                    
    return T_tilde_block, V_tilde_block

# --- Top-level wrapper for joblib ---
def _task_wrapper_for_joblib_top_level(args_tuple_joblib):
    (idx1_jl, k1_tuple_jl, idx2_jl, k2_tuple_jl,
     data_k1_jl, data_k2_jl,
     r_model_jl, rho_model_jl, g_model_jl, kappa_model_jl, mu_model_jl, r_stable_jl,
     G_const_jl,
     perturbations_direct_coeffs_jl,
     s_max_jl,
     l1_jl, l2_jl, m_values_l1_jl, m_values_l2_jl, matrix_size_l1m_jl, matrix_size_l2m_jl
    ) = args_tuple_joblib

    T_block_res, V_block_res = _calculate_k1k2_tilde_block_job_top_level(
        k1_tuple_jl, k2_tuple_jl,
        data_k1_jl, data_k2_jl,
        r_model_jl, rho_model_jl, g_model_jl, kappa_model_jl, mu_model_jl, r_stable_jl,
        G_const_jl,
        perturbations_direct_coeffs_jl,
        s_max_jl,
        l1_jl, l2_jl, m_values_l1_jl, m_values_l2_jl, matrix_size_l1m_jl, matrix_size_l2m_jl
    )
    return idx1_jl, idx2_jl, T_block_res, V_block_res

class MoonPerturbationCalculatorGeneralized:
    def __init__(self, 
                 all_spher_nl_tuples, 
                 all_tor_nl_tuples,   
                 full_omega_k2_map, 
                 model_file,
                 eigenfunction_folder_spher, eigenfunction_folder_tor,
                 perturbations_input_dictionary, s_max=8):
        
        self.perturbations_direct_coeffs = perturbations_input_dictionary
        self.s_max = s_max
        #self.G = Const_G # Store G in self for easy access
        self.G = 0

        print(f"\n--- Initializing Generalized Calculator for ALL specified L values ---")

        print(f"Loading background model from: {model_file}")
        self._load_background_model(model_file)
        self.R_moon = self.r_model[-1]
        print(f"Model loaded: {len(self.r_model)} radial points, R = {self.R_moon / 1e3:.1f} km")

        self.eigen_data = {}
        self.k_values_with_data = [] 
        
        potential_k_tuples = []
        for n_val, l_val in sorted(list(set(all_spher_nl_tuples))):
            potential_k_tuples.append(('S', n_val, l_val))
        for n_val, l_val in sorted(list(set(all_tor_nl_tuples))):
            if l_val > 0: 
                potential_k_tuples.append(('T', n_val, l_val))
        
        print("Validating modes (checking for frequency and eigenfunction files)...\n")
        for k_tuple in potential_k_tuples:
            sigma_char, n_val, l_val = k_tuple
            if k_tuple not in full_omega_k2_map:
                print(f"  Skipping {k_tuple}: Frequency not found in provided omega_k2_map.")
                continue
            if full_omega_k2_map[k_tuple] <= 1e-20:
                 print(f"  Skipping {k_tuple}: Non-positive omega_k^2 ({full_omega_k2_map[k_tuple]:.2e}) found.")
                 continue
            if l_val == 0 and sigma_char == 'T':
                 print(f"  Skipping {k_tuple}: Toroidal modes not physical for l=0.")
                 continue
            try:
                self._check_eigenfunction_file_exists(k_tuple, eigenfunction_folder_spher, eigenfunction_folder_tor)
                self.k_values_with_data.append(k_tuple)
            except FileNotFoundError as e:
                print(f"  Skipping {k_tuple}: Eigenfunction file not found ({e}).")
        
        self.k_values_with_data.sort(key=lambda x: (x[2], x[0] == 'T', x[1])) 
        
        if not self.k_values_with_data:
            print(f"Warning: No valid modes found with both frequency and eigenfunction data. Aborting.")
            self.num_k_modes = 0 
            return 

        self.k_map = {k_tuple: i for i, k_tuple in enumerate(self.k_values_with_data)}
        self.num_k_modes = len(self.k_values_with_data)
        self.l_values_present = sorted(list(set(k[2] for k in self.k_values_with_data)))
        print(f"\nFound {self.num_k_modes} valid modes across L values {self.l_values_present} to be processed.")

        self.omega_k2_map_instance = {k: full_omega_k2_map[k] for k in self.k_values_with_data if k in full_omega_k2_map}
        self._prepare_all_eigenfunctions(eigenfunction_folder_spher, eigenfunction_folder_tor)

        self.results_delta_omega_sq = {} 
        
        self.V_tilde_kk_mm = {} 
        self.T_tilde_kk_mm = {}
        self.V_real_kk_mm = {}
        self.T_real_kk_mm = {}

    def _get_k_ang(self, l_val):
        return np.sqrt(l_val * (l_val + 1)) if l_val > 0 else 1e-9 

    def _get_m_values(self, l_val):
        return np.arange(-l_val, l_val + 1)

    def _get_matrix_size_lm(self, l_val):
        return 2 * l_val + 1

    def _load_background_model(self, modname):
        model = np.loadtxt(modname, skiprows=0)
        self.r_model = model[:, 0]
        self.rho_model = model[:, 1]
        self.vp_model = model[:, 2]
        self.vs_model = model[:, 3]
        self.mu_model = self.rho_model * self.vs_model**2
        self.kappa_model = self.rho_model * (self.vp_model**2 - (4.0/3.0) * self.vs_model**2)
        self.kappa_model[self.kappa_model < 0] = 0
        self.r_stable = np.maximum(self.r_model, 1e-10) 
        mass_integrand = self.rho_model * self.r_model**2
        mass_r = 4 * np.pi * self._cumulative_integrate(mass_integrand, self.r_model)
        self.g_model = np.divide(-self.G * mass_r, self.r_stable**2, out=np.zeros_like(self.r_model), where=self.r_stable!=0)
        if len(self.g_model)>0: self.g_model[0] = 0.0

    def _get_eigen_fname_path(self, k_tuple, eigenfunction_folder_spher, eigenfunction_folder_tor):
        sigma_char, n_val, l_val = k_tuple
        n_str_formatted = f"{n_val:07d}"
        l_str_formatted = f"{l_val:07d}" 
        base_folder = eigenfunction_folder_spher if sigma_char == 'S' else eigenfunction_folder_tor
        prefix = "S." if sigma_char == 'S' else "T."
        filename_path = os.path.join(base_folder, f"{prefix}{n_str_formatted}.{l_str_formatted}.ASC")
        if not os.path.exists(filename_path):
             raise FileNotFoundError(filename_path)
        return filename_path

    def _check_eigenfunction_file_exists(self, k_tuple, eigenfunction_folder_spher, eigenfunction_folder_tor):
        self._get_eigen_fname_path(k_tuple, eigenfunction_folder_spher, eigenfunction_folder_tor)

    def _load_single_eigenfunction(self, k_tuple, eigenfunction_folder_spher, eigenfunction_folder_tor):
        sigma_char, n_val, l_val = k_tuple
        k_ang_l = self._get_k_ang(l_val)
        filename = self._get_eigen_fname_path(k_tuple, eigenfunction_folder_spher, eigenfunction_folder_tor)
        
        df = pd.read_csv(filename, skiprows=1, header=None, delim_whitespace=True)
        
        if sigma_char == 'S':
            U_M_raw = np.array(df[1], dtype=float)[::-1] 
            V_M_raw = np.array(df[3], dtype=float)[::-1]

            if len(U_M_raw) != len(self.r_model):
                raise ValueError(f"Length mismatch for S-mode {k_tuple} U^M. File: {filename}.")
            if len(V_M_raw) != len(self.r_model):
                raise ValueError(f"Length mismatch for S-mode {k_tuple} V^M. File: {filename}.")

            if l_val == 0:
                norm_integrand = self.rho_model * (U_M_raw**2) * self.r_model**2
                V_M_raw = np.zeros_like(V_M_raw) 
            else:
                norm_integrand = self.rho_model * \
                                 (U_M_raw**2 + k_ang_l**2 * V_M_raw**2) * \
                                 self.r_model**2
            
            norm_factor_val_sq = self._integrate(norm_integrand, self.r_model)

            if norm_factor_val_sq < 1e-20:
                print(f"Warning: S-mode Normalization factor squared for {k_tuple} is very small ({norm_factor_val_sq}). Using 1e10 for sqrt(norm).")
                norm_factor_sqrt = 1e10 
            else:
                norm_factor_sqrt = np.sqrt(norm_factor_val_sq)
            
            U_latex = U_M_raw / norm_factor_sqrt
            V_latex = (k_ang_l * V_M_raw) / norm_factor_sqrt
            if l_val == 0: V_latex = np.zeros_like(V_latex)

            return {'U_latex': U_latex, 'V_latex': V_latex}

        elif sigma_char == 'T': 
            W_M_raw = np.array(df[1], dtype=float)[::-1]

            if len(W_M_raw) != len(self.r_model):
                raise ValueError(f"Length mismatch for T-mode {k_tuple}. File: {filename}.")

            if l_val == 0: 
                raise ValueError(f"Toroidal mode T {k_tuple} requested for l=0, which is not physical.")
            
            W_true_unnorm = W_M_raw
            norm_integrand = self.rho_model * W_true_unnorm**2 * self.r_model**2
            norm_factor_sq = self._integrate(norm_integrand, self.r_model)

            if norm_factor_sq < 1e-20:
                print(f"Warning: T-mode Normalization factor squared for {k_tuple} is very small ({norm_factor_sq}). Using 1e10 for sqrt(norm).")
                norm_factor_sqrt = 1e10
            else:
                norm_factor_sqrt = np.sqrt(norm_factor_sq)
            
            W_latex = W_true_unnorm / norm_factor_sqrt
            return {'W_latex': W_latex}

    def _process_single_eigenfunction_job(self, k_tuple_job, eigenfunction_folder_spher_job, eigenfunction_folder_tor_job):
        sigma_char, n_val, l_val = k_tuple_job
        k_ang_l = self._get_k_ang(l_val)
        
        raw_efuncs_latex = self._load_single_eigenfunction(k_tuple_job, eigenfunction_folder_spher_job, eigenfunction_folder_tor_job)
        data = {'sigma': sigma_char, 'n': n_val, 'l': l_val}

        if sigma_char == 'S':
            data['u'] = raw_efuncs_latex['U_latex']
            data['v_div_k'] = np.divide(raw_efuncs_latex['V_latex'], k_ang_l, 
                                        out=np.zeros_like(raw_efuncs_latex['V_latex']), where=k_ang_l!=0)
            
            data['dot_u'] = self._derivative(data['u'], self.r_model)
            data['dot_v_div_k'] = self._derivative(data['v_div_k'], self.r_model)
            
            data['f_val'] = np.divide(2*data['u'] - k_ang_l**2 * data['v_div_k'], self.r_stable, 
                                      out=np.zeros_like(self.r_model), where=self.r_stable!=0)
            
            term_v_div_k_r = np.divide(data['v_div_k'], self.r_stable, out=np.zeros_like(self.r_model), where=self.r_stable!=0)
            term_u_r = np.divide(data['u'], self.r_stable, out=np.zeros_like(self.r_model), where=self.r_stable!=0)
            data['x_val'] = data['dot_v_div_k'] - term_v_div_k_r + term_u_r
            
            V_for_P_calc = raw_efuncs_latex['V_latex']
            
            integrand1_p = self.rho_model * (l_val * data['u'] + V_for_P_calc) * self.r_stable**(l_val + 1) 
            integrand2_p = self.rho_model * (-(l_val + 1) * data['u'] + V_for_P_calc) * \
                           np.power(self.r_stable, -l_val, out=np.zeros_like(self.r_model), where=self.r_stable!=0)
            
            int1_p = self._cumulative_integrate(integrand1_p, self.r_model)
            int2_total_p = self._integrate(integrand2_p, self.r_model)
            int2_cumulative_p = self._cumulative_integrate(integrand2_p, self.r_model)
            int2_rev_p = int2_total_p - int2_cumulative_p
            
            P_term1 = np.divide(int1_p, np.power(self.r_stable, l_val + 1, out=np.ones_like(self.r_model), where=self.r_stable!=0), 
                                out=np.zeros_like(self.r_model), where=self.r_stable!=0)
            P_term2 = int2_rev_p * np.power(self.r_stable, l_val, out=np.zeros_like(self.r_model), where=self.r_stable!=0)
            
            data['p_val'] = -4 * np.pi * self.G / (2*l_val + 1) * (P_term1 + P_term2) # Use self.G
            if len(data['p_val']) > 0: data['p_val'][0] = 0.0 
            data['dot_p'] = self._derivative(data['p_val'], self.r_model)

        elif sigma_char == 'T':
            data['w_div_k'] = np.divide(raw_efuncs_latex['W_latex'], k_ang_l,
                                        out=np.zeros_like(raw_efuncs_latex['W_latex']), where=k_ang_l!=0)
            data['dot_w_div_k'] = self._derivative(data['w_div_k'], self.r_model)
            term_w_div_k_r = np.divide(data['w_div_k'], self.r_stable, out=np.zeros_like(self.r_model), where=self.r_stable!=0)
            data['z_val'] = data['dot_w_div_k'] - term_w_div_k_r
        
        return k_tuple_job, data

    def _prepare_all_eigenfunctions(self, eigenfunction_folder_spher, eigenfunction_folder_tor):
        print("Preparing all eigenfunction derivatives and related quantities for valid modes (parallelized)...\n")
        
        tasks = [delayed(self._process_single_eigenfunction_job)(k_tuple, eigenfunction_folder_spher, eigenfunction_folder_tor)
                 for k_tuple in self.k_values_with_data]
        
        results = Parallel(n_jobs=-2, backend='loky', verbose=10)(tasks) 

        for k_tuple_res, data_res in results:
            self.eigen_data[k_tuple_res] = data_res
        
        print(f"\nAll {len(self.k_values_with_data)} eigenfunctions processed and collected.")

    def _derivative(self, y, x):
        return np.gradient(y, x, edge_order=2)

    def _integrate(self, y, x): 
        return simps(y, x)

    def _cumulative_integrate(self, y, x, initial=0): 
        return cumulative_trapezoid(y, x, initial=initial)

    def _calculate_tilde_matrices(self):
        if self.num_k_modes == 0: return
        print("Calculating tilde V and T matrices (parallelized)...\n")

        job_args = []
        for idx1, k1_tuple in enumerate(self.k_values_with_data):
            s1_char, n1, l1 = k1_tuple
            m_values_l1 = self._get_m_values(l1) 
            matrix_size_l1m = self._get_matrix_size_lm(l1)

            for idx2, k2_tuple in enumerate(self.k_values_with_data):
                s2_char, n2, l2 = k2_tuple
                m_values_l2 = self._get_m_values(l2)
                matrix_size_l2m = self._get_matrix_size_lm(l2)

                job_args.append((
                    idx1, k1_tuple, idx2, k2_tuple,
                    self.eigen_data[k1_tuple], self.eigen_data[k2_tuple],
                    self.r_model, self.rho_model, self.g_model, self.kappa_model, self.mu_model, self.r_stable,
                    self.G, 
                    self.perturbations_direct_coeffs,
                    self.s_max,
                    l1, l2, m_values_l1, m_values_l2, matrix_size_l1m, matrix_size_l2m
                ))
        
        tasks = [delayed(_task_wrapper_for_joblib_top_level)(args) for args in job_args]
        results_parallel = Parallel(n_jobs=-2, backend='loky', verbose=10)(tasks)

        for idx1_res, idx2_res, T_block_val, V_block_val in results_parallel:
            self.T_tilde_kk_mm[(idx1_res, idx2_res)] = T_block_val
            self.V_tilde_kk_mm[(idx1_res, idx2_res)] = V_block_val
        
        print("\nFinished calculating tilde matrices.")

    def _convert_tilde_to_real_matrices(self):
        if self.num_k_modes == 0: return
        print("Converting tilde matrices to real basis...")

        for idx1, k1_tuple in enumerate(self.k_values_with_data):
            l1 = k1_tuple[2]
            m_values_l1 = self._get_m_values(l1)
            matrix_size_l1m = self._get_matrix_size_lm(l1)

            for idx2, k2_tuple in enumerate(self.k_values_with_data):
                l2 = k2_tuple[2]
                m_values_l2 = self._get_m_values(l2)
                matrix_size_l2m = self._get_matrix_size_lm(l2)

                M_tilde_T_block_curr = self.T_tilde_kk_mm[(idx1, idx2)] 
                M_tilde_V_block_curr = self.V_tilde_kk_mm[(idx1, idx2)] 
                
                M_real_T_block_curr = np.zeros((matrix_size_l1m, matrix_size_l2m), dtype=complex)
                M_real_V_block_curr = np.zeros((matrix_size_l1m, matrix_size_l2m), dtype=complex)

                def get_tilde_elem_local(tilde_matrix_local, m_val_for_row, m_val_for_col):
                    row_idx = int(m_val_for_row + l1)
                    col_idx = int(m_val_for_col + l2)
                    return tilde_matrix_local[row_idx, col_idx]

                for r_idx_loop, m_row_real_val_orig in enumerate(m_values_l1): 
                    m_row_real_val = int(m_row_real_val_orig)
                    for c_idx_loop, m_col_real_val_orig in enumerate(m_values_l2): 
                        m_col_real_val = int(m_col_real_val_orig)
                        
                        val_T_curr, val_V_curr = 0.0j, 0.0j
                        
                        m_abs_row = abs(m_row_real_val) 
                        m_abs_col = abs(m_col_real_val) 

                        if m_row_real_val == 0 and m_col_real_val == 0: 
                            val_T_curr = get_tilde_elem_local(M_tilde_T_block_curr, 0, 0)
                            val_V_curr = get_tilde_elem_local(M_tilde_V_block_curr, 0, 0)
                        elif m_row_real_val == 0: 
                            if m_col_real_val > 0: 
                                val_T_curr = np.sqrt(2) * np.imag(get_tilde_elem_local(M_tilde_T_block_curr, 0, m_abs_col))
                                val_V_curr = np.sqrt(2) * np.imag(get_tilde_elem_local(M_tilde_V_block_curr, 0, m_abs_col))
                            else: 
                                val_T_curr = np.sqrt(2) * np.real(get_tilde_elem_local(M_tilde_T_block_curr, 0, m_abs_col))
                                val_V_curr = np.sqrt(2) * np.real(get_tilde_elem_local(M_tilde_V_block_curr, 0, m_abs_col))
                        elif m_col_real_val == 0: 
                            if m_row_real_val > 0: 
                                val_T_curr = -np.sqrt(2) * np.imag(get_tilde_elem_local(M_tilde_T_block_curr, m_abs_row, 0))
                                val_V_curr = -np.sqrt(2) * np.imag(get_tilde_elem_local(M_tilde_V_block_curr, m_abs_row, 0))
                            else: 
                                val_T_curr = np.sqrt(2) * np.real(get_tilde_elem_local(M_tilde_T_block_curr, m_abs_row, 0))
                                val_V_curr = np.sqrt(2) * np.real(get_tilde_elem_local(M_tilde_V_block_curr, m_abs_row, 0))
                        else: 
                            term_T_mmprime_tilde = get_tilde_elem_local(M_tilde_T_block_curr, m_abs_row, m_abs_col)
                            term_V_mmprime_tilde = get_tilde_elem_local(M_tilde_V_block_curr, m_abs_row, m_abs_col)
                            term_T_m_neg_mprime_tilde = get_tilde_elem_local(M_tilde_T_block_curr, m_abs_row, -m_abs_col)
                            term_V_m_neg_mprime_tilde = get_tilde_elem_local(M_tilde_V_block_curr, m_abs_row, -m_abs_col)
                            
                            sign_m_prime_parity = (-1)**m_abs_col 

                            if m_row_real_val > 0 and m_col_real_val > 0: 
                                val_T_curr = np.real(term_T_mmprime_tilde - sign_m_prime_parity * term_T_m_neg_mprime_tilde)
                                val_V_curr = np.real(term_V_mmprime_tilde - sign_m_prime_parity * term_V_m_neg_mprime_tilde)
                            elif m_row_real_val > 0 and m_col_real_val < 0: 
                                val_T_curr = -np.imag(term_T_mmprime_tilde + sign_m_prime_parity * term_T_m_neg_mprime_tilde)
                                val_V_curr = -np.imag(term_V_mmprime_tilde + sign_m_prime_parity * term_V_m_neg_mprime_tilde)
                            elif m_row_real_val < 0 and m_col_real_val > 0: 
                                val_T_curr = np.imag(term_T_mmprime_tilde - sign_m_prime_parity * term_T_m_neg_mprime_tilde) 
                                val_V_curr = np.imag(term_V_mmprime_tilde - sign_m_prime_parity * term_V_m_neg_mprime_tilde) 
                            elif m_row_real_val < 0 and m_col_real_val < 0: 
                                val_T_curr = np.real(term_T_mmprime_tilde + sign_m_prime_parity * term_T_m_neg_mprime_tilde)
                                val_V_curr = np.real(term_V_mmprime_tilde + sign_m_prime_parity * term_V_m_neg_mprime_tilde)
                        
                        M_real_T_block_curr[r_idx_loop, c_idx_loop] = val_T_curr
                        M_real_V_block_curr[r_idx_loop, c_idx_loop] = val_V_curr
                
                self.T_real_kk_mm[(idx1, idx2)] = M_real_T_block_curr.real 
                self.V_real_kk_mm[(idx1, idx2)] = M_real_V_block_curr.real
        
        print("Finished converting to real basis matrices.")

    def _calculate_delta_omega_sq(self):
        if self.num_k_modes == 0: return
        print("Calculating delta_omega_k_sq...")
        for idx, k_tuple in enumerate(self.k_values_with_data):
            l_k = k_tuple[2]
            matrix_size_lk_m = self._get_matrix_size_lm(l_k)
            nan_array_lm = np.full(matrix_size_lk_m, np.nan + 1j*np.nan)

            try: omega_k_sq = self.omega_k2_map_instance[k_tuple]
            except KeyError:
                print(f"Error: omega_k^2 not found for k={k_tuple} in instance map. Skipping delta_omega_sq.")
                self.results_delta_omega_sq[idx] = nan_array_lm
                continue
            
            V_kk_block = self.V_real_kk_mm[(idx, idx)]; T_kk_block = self.T_real_kk_mm[(idx, idx)]
            W_k_matrix_curr = V_kk_block - omega_k_sq * T_kk_block 
            
            try:
                eigvals, _ = np.linalg.eig(W_k_matrix_curr) 
                sort_indices = np.argsort(np.real(eigvals))
                self.results_delta_omega_sq[idx] = eigvals[sort_indices]
            except np.linalg.LinAlgError as e:
                print(f"Error diagonalizing W_k for k={k_tuple}: {e}")
                self.results_delta_omega_sq[idx] = nan_array_lm
                continue 
        print("Finished delta_omega_k_sq.")
        
    def run_calculation(self):
        if self.num_k_modes == 0:
            print(f"No valid modes to process. Calculation run aborted.")
            return
        self._calculate_tilde_matrices()
        self._convert_tilde_to_real_matrices()
        self._calculate_delta_omega_sq()
        print(f"All calculations finished for all specified L values.")

    def save_results(self, output_folder_base): 
        if self.num_k_modes == 0:
            print(f"No results to save as no valid modes were processed.")
            return
            
        output_folder_unified = Path(output_folder_base)
        print(f"Saving results to unified files in: {output_folder_unified}")
        output_folder_unified.mkdir(parents=True, exist_ok=True)

        d_omega_filename_unified = output_folder_unified / "ALL_L_delta_omega_km.txt"
        t_real_filename_unified = output_folder_unified / "ALL_L_T_real_matrix.txt"
        v_real_filename_unified = output_folder_unified / "ALL_L_V_real_matrix.txt"

        with open(d_omega_filename_unified, 'w') as f_om:
            f_om.write("# Unified delta_omega_km (frequency shifts) for all processed modes (l,m)\n")
            f_om.write("# delta_omega_km = delta_omega_km_sq / (2 * omega_k_unperturbed)\n")
            f_om.write("# sigma_char,n_val,l_val,m_perturbed_state_idx,Re(delta_omega_km),Im(delta_omega_km)\n")
            
            for k_idx_map_val, k_tuple_val in enumerate(self.k_values_with_data):
                sigma_char_val, n_val_val, l_val_k_val = k_tuple_val
                
                delta_omega_km_sq_values_arr = self.results_delta_omega_sq.get(k_idx_map_val)
                if delta_omega_km_sq_values_arr is None: continue

                try:
                    omega_k_unperturbed_sq_val = self.omega_k2_map_instance[k_tuple_val]
                except KeyError:
                    print(f"Warning: Omega_k_sq for {k_tuple_val} not found in instance map during save. Skipping delta_omega for this k.")
                    continue
                
                nan_array_like = np.full_like(delta_omega_km_sq_values_arr, np.nan + 1j*np.nan, dtype=complex)
                if omega_k_unperturbed_sq_val <= 1e-20: 
                    delta_omega_km_values_arr = nan_array_like
                else:
                    omega_k_unperturbed_val = np.sqrt(omega_k_unperturbed_sq_val)
                    valid_mask = ~np.isnan(delta_omega_km_sq_values_arr) # Should be all valid now
                    delta_omega_km_values_arr = nan_array_like.copy() 
                    delta_omega_km_values_arr[valid_mask] = delta_omega_km_sq_values_arr[valid_mask] / (2 * omega_k_unperturbed_val)

                for m_idx_perturbed_state in range(len(delta_omega_km_values_arr)):
                    d_om_val_loop = delta_omega_km_values_arr[m_idx_perturbed_state]
                    if np.isnan(d_om_val_loop.real) or np.isnan(d_om_val_loop.imag):
                        # This might still happen if omega_k_unperturbed_sq_val was too small
                        # or if linalg.eig returned NaNs for some reason (though errors are caught)
                        f_om.write(f"{sigma_char_val},{n_val_val},{l_val_k_val},{m_idx_perturbed_state},nan,nan\n")
                        continue 
                    f_om.write(f"{sigma_char_val},{n_val_val},{l_val_k_val},{m_idx_perturbed_state},{d_om_val_loop.real:.8e},{d_om_val_loop.imag:.8e}\n")
        print(f"delta_omega_km saved to {d_omega_filename_unified}")

        with open(t_real_filename_unified, 'w') as f_t_real:
            f_t_real.write("# Unified Real Basis Kinetic Energy Matrix Elements T_k1k2\n")
            f_t_real.write("# T_k1k2 is stored as T_real_kk_mm[(idx1, idx2)]\n")
            f_t_real.write("# sigma1,n1,l1,sigma2,n2,l2,m1_row_idx(for l1),m2_col_idx(for l2),Re(T_val),Im(T_val) (Im(T_val) is 0.0)\n")
            
            for idx1_row_loop, k1_tuple_row_loop in enumerate(self.k_values_with_data):
                s1_char_loop, n1_loop, l1_val_loop = k1_tuple_row_loop
                matrix_size_l1m = self._get_matrix_size_lm(l1_val_loop)

                for idx2_col_loop, k2_tuple_col_loop in enumerate(self.k_values_with_data):
                    s2_char_loop, n2_loop, l2_val_loop = k2_tuple_col_loop
                    matrix_size_l2m = self._get_matrix_size_lm(l2_val_loop)
                    
                    T_block_curr = self.T_real_kk_mm.get((idx1_row_loop, idx2_col_loop))
                    if T_block_curr is None: continue # Should not happen if calculation ran fully

                    for i_row_m1 in range(matrix_size_l1m): 
                        for j_col_m2 in range(matrix_size_l2m): 
                            val_to_write = T_block_curr[i_row_m1,j_col_m2] # This is a real float
                            if np.isnan(val_to_write): # Check if it's NaN (e.g. if an error occurred upstream)
                                f_t_real.write(f"{s1_char_loop},{n1_loop},{l1_val_loop},{s2_char_loop},{n2_loop},{l2_val_loop},{i_row_m1},{j_col_m2},nan,nan\n")
                                continue
                            f_t_real.write(f"{s1_char_loop},{n1_loop},{l1_val_loop},{s2_char_loop},{n2_loop},{l2_val_loop},{i_row_m1},{j_col_m2},{val_to_write:.8e},0.0e+00\n")
        print(f"T_real_kk_mm saved to {t_real_filename_unified}")

        with open(v_real_filename_unified, 'w') as f_v_real:
            f_v_real.write("# Unified Real Basis Potential Energy Matrix Elements V_k1k2\n")
            f_v_real.write("# V_k1k2 is stored as V_real_kk_mm[(idx1, idx2)]\n")
            f_v_real.write("# sigma1,n1,l1,sigma2,n2,l2,m1_row_idx(for l1),m2_col_idx(for l2),Re(V_val),Im(V_val) (Im(V_val) is 0.0)\n")
            
            for idx1_row_loop, k1_tuple_row_loop in enumerate(self.k_values_with_data):
                s1_char_loop, n1_loop, l1_val_loop = k1_tuple_row_loop
                matrix_size_l1m = self._get_matrix_size_lm(l1_val_loop)

                for idx2_col_loop, k2_tuple_col_loop in enumerate(self.k_values_with_data):
                    s2_char_loop, n2_loop, l2_val_loop = k2_tuple_col_loop
                    matrix_size_l2m = self._get_matrix_size_lm(l2_val_loop)
                    
                    V_block_curr = self.V_real_kk_mm.get((idx1_row_loop, idx2_col_loop))
                    if V_block_curr is None: continue

                    for i_row_m1 in range(matrix_size_l1m): 
                        for j_col_m2 in range(matrix_size_l2m): 
                            val_to_write = V_block_curr[i_row_m1,j_col_m2] # This is a real float
                            if np.isnan(val_to_write):
                                f_v_real.write(f"{s1_char_loop},{n1_loop},{l1_val_loop},{s2_char_loop},{n2_loop},{l2_val_loop},{i_row_m1},{j_col_m2},nan,nan\n")
                                continue
                            f_v_real.write(f"{s1_char_loop},{n1_loop},{l1_val_loop},{s2_char_loop},{n2_loop},{l2_val_loop},{i_row_m1},{j_col_m2},{val_to_write:.8e},0.0e+00\n")
        print(f"V_real_kk_mm saved to {v_real_filename_unified}")

In [5]:
# Cell 3: Main Script Example (Adapted for Global Calculator)
if __name__ == "__main__":
    print("Setting up Full Generalized Perturbation Calculation Example (Simplified)...")

    # --- RANGES (USER: Adapt these) ---
    L_MIN_SPHER_GLOBAL = 2
    L_MAX_SPHER_GLOBAL = 2 
    N_MIN_SPHER_GLOBAL = 0
    N_MAX_SPHER_GLOBAL = 200

    L_MIN_TOR_GLOBAL = 0 
    L_MAX_TOR_GLOBAL = 0 # Set to 0 if no toroidal, or e.g. 2 if L=2 toroidal needed
    N_MIN_TOR_GLOBAL = 0
    N_MAX_TOR_GLOBAL = 0 # Set to 0 if no toroidal

    # --- Prepare lists of (n,l) tuples for the calculator ---
    all_spher_nl_tuples_main = []
    if L_MAX_SPHER_GLOBAL >= L_MIN_SPHER_GLOBAL and N_MAX_SPHER_GLOBAL >= N_MIN_SPHER_GLOBAL :
        for l_spher in range(L_MIN_SPHER_GLOBAL, L_MAX_SPHER_GLOBAL + 1):
            for n_spher in range(N_MIN_SPHER_GLOBAL, N_MAX_SPHER_GLOBAL + 1):
                all_spher_nl_tuples_main.append((n_spher, l_spher))
    
    all_tor_nl_tuples_main = []
    if L_MAX_TOR_GLOBAL >= L_MIN_TOR_GLOBAL and N_MAX_TOR_GLOBAL >= N_MIN_TOR_GLOBAL:
        for l_tor in range(L_MIN_TOR_GLOBAL, L_MAX_TOR_GLOBAL + 1):
            if l_tor == 0: continue # Toroidal not for l=0
            for n_tor in range(N_MIN_TOR_GLOBAL, N_MAX_TOR_GLOBAL + 1):
                all_tor_nl_tuples_main.append((n_tor, l_tor))
    
    if not all_spher_nl_tuples_main and not all_tor_nl_tuples_main:
        print("Warning: No (n,l) mode combinations specified for processing. Exiting.")
        # sys.exit() # Or handle as appropriate

    # --- PATHS (USER: Adapt these to your file structure) ---\n",
    modename = "Reference_Model" # Example mode name, can be used in paths

    spheroidal_data_base_dir_main = Path("D:\\Study\\Research & Survey\\Seismic GW detector\\MyWork\\Inverse-1D-pert\\MultModel_Results1024\\"+modename) 
    eigen_freq_file_s_main = spheroidal_data_base_dir_main / "res1.txt"
    eigen_func_folder_s_main = spheroidal_data_base_dir_main / "db25new" 
    
    toroidal_data_base_dir_main = Path("D:\\Study\\Research & Survey\\Seismic GW detector\\MyWork\\LunarResponse\\EigenCalculation\\"+modename+"_toroidal")
    eigen_freq_file_t_main = toroidal_data_base_dir_main / "res1.txt"
    eigen_func_folder_t_main = toroidal_data_base_dir_main / "db25new" 
    
    model_file_path_main = Path("D:\\Study\\Research & Survey\\Seismic GW detector\\MyWork\\Inverse-1D-pert\\MultModel_Results1024\\"+modename+"\\"+modename+"_8000_code.txt")
    
    # Output folder name can still reflect the overall ranges
    s_l_range_str = f"{N_MIN_SPHER_GLOBAL}S{L_MIN_SPHER_GLOBAL}-{N_MAX_SPHER_GLOBAL}S{L_MAX_SPHER_GLOBAL}" if all_spher_nl_tuples_main else "NoSpher"
    t_l_range_str = f"{N_MIN_TOR_GLOBAL}T{L_MIN_TOR_GLOBAL}-{N_MAX_TOR_GLOBAL}T{L_MAX_TOR_GLOBAL}" if all_tor_nl_tuples_main else "NoTor"
    # Changed output folder name slightly to indicate simplified results
    output_folder_base_main = Path(f"TV_{s_l_range_str}_{t_l_range_str}_matrix")
    # --- END PATHS ---

    # --- Load Frequencies ---
    full_omega_k2_map_from_file = {}
    def _load_frequencies_from_res1_helper(res1_path, sigma_char, target_map, 
                                           nl_tuples_to_consider): # nl_tuples_to_consider is list of (n,l)
        print(f"Attempting to load {sigma_char}-mode frequencies from: {res1_path}")
        # Create a set of (n,l) for faster lookup if nl_tuples_to_consider is large
        nl_set_to_consider = set(nl_tuples_to_consider)
        if not nl_set_to_consider:
            print(f"  No {sigma_char}-modes specified for loading frequencies.")
            return

        if res1_path.exists():
            try:
                # Original code used np.loadtxt(res1_path) which implies space/tab delimited
                # and assumes header might be handled by skiprows or comments
                # For standard res1.txt, often first line is header
                data_freq = np.loadtxt(res1_path, comments='#') # Use comments='#' or skiprows=1
                if data_freq.ndim == 1: data_freq = data_freq.reshape(1, -1) 
                for row in data_freq:
                    # Original code used row[0], row[1], row[3]
                    # n l T Q f
                    # 0 1 2 3 4  <- indices
                    # Assuming freq_mhz = row[4] if 5 columns, or row[3] if 4 columns (like in original)
                    # Let's stick to the original interpretation: n, l, (some T/Q), f
                    n_file, l_file = int(row[0]), int(row[1])
                    if (n_file, l_file) not in nl_set_to_consider: continue 
                    
                    freq_mhz = float(row[3]) # As per original example's interpretation of res1.txt
                    omega_sq_val = (freq_mhz * 1e-3 * 2 * np.pi)**2
                    if omega_sq_val > 1e-20: 
                        target_map[(sigma_char, n_file, l_file)] = omega_sq_val
                    else:
                        print(f"  Warning: Skipping zero/negative omega_sq ({omega_sq_val:.2e}) for ({sigma_char},{n_file},{l_file}) from file {res1_path}")
                print(f"  Successfully loaded relevant {sigma_char}-mode frequencies from {res1_path}")
            except Exception as e:
                print(f"  Error parsing {res1_path}: {e}. Frequencies for {sigma_char}-modes might be missing or incomplete.")
        else:
            print(f"  Warning: Eigenfrequency file '{res1_path}' for {sigma_char}-modes not found.")
        
    _load_frequencies_from_res1_helper(eigen_freq_file_s_main, 'S', full_omega_k2_map_from_file, all_spher_nl_tuples_main)
    _load_frequencies_from_res1_helper(eigen_freq_file_t_main, 'T', full_omega_k2_map_from_file, all_tor_nl_tuples_main)
    print(f"Total {len(full_omega_k2_map_from_file)} frequency entries compiled for specified (n,l) ranges.")

    # --- Dummy Model File (if needed, using original logic for checking existence) ---\n",
    r_surface_moon_dummy = 1737.1e3 # Used if creating dummy
    ''' # This block is from the original, kept for structural similarity, but usually real file is expected
    if not model_file_path_main.exists():
        print(f"Creating dummy model file: {model_file_path_main}")
        dummy_r_pts = np.linspace(0,r_surface_moon_dummy,200); dummy_rho_pts = np.full_like(dummy_r_pts,3340.0)
        dummy_vp_pts = np.full_like(dummy_r_pts,5000.0); dummy_vs_pts = np.full_like(dummy_r_pts,3000.0)
        dummy_vs_pts[dummy_r_pts < 0.3*r_surface_moon_dummy]=0 # Simple core with Vs=0
        np.savetxt(model_file_path_main,np.array([dummy_r_pts,dummy_rho_pts,dummy_vp_pts,dummy_vs_pts]).T,fmt="%.6e",header="r rho vp vs")
    '''
    # --- Perturbations (EXACTLY AS IN ORIGINAL) ---
    s_perturbation_degree_main = 1 
    delta_d_perturbations_main = {
(352500, 0, 0): 352500*0.002,
                                  }
    delta_rho_perturbations_main = {

                                    }
    
    
    delta_kappa_perturbations_main = {


                                    }
    
    delta_mu_perturbations_main = {

                                    }
    
    PERTURBATION_SCALE = 1  # 例如放大2倍，缩小为0.5倍，按需设置

    # 缩放 delta_d_perturbations_main
    for k in delta_d_perturbations_main:
        delta_d_perturbations_main[k] *= PERTURBATION_SCALE

    # 缩放 delta_rho_perturbations_main
    for k in delta_rho_perturbations_main:
        v = delta_rho_perturbations_main[k]
        if isinstance(v, dict) and 'value' in v:
            v['value'] *= PERTURBATION_SCALE
        elif isinstance(v, (int, float)):
            delta_rho_perturbations_main[k] *= PERTURBATION_SCALE

    all_perturbations_main = {'delta_rho':delta_rho_perturbations_main,'delta_mu':delta_mu_perturbations_main,'delta_kappa':delta_kappa_perturbations_main, 'delta_d':delta_d_perturbations_main}
    print(f"Defined example perturbations up to s_max={s_perturbation_degree_main}")
    
    output_folder_base_main.mkdir(parents=True, exist_ok=True)

    # --- Instantiate and Run Calculator (once for all modes) ---
    # Threshold parameters removed from instantiation
    if not all_spher_nl_tuples_main and not all_tor_nl_tuples_main:
        print("\nNo modes specified. Skipping calculation.")
    elif not model_file_path_main.exists(): # Check if model file exists before proceeding
        print(f"\nModel file {model_file_path_main} not found. Skipping calculation.")
    elif not full_omega_k2_map_from_file and (all_spher_nl_tuples_main or all_tor_nl_tuples_main) : # Check if freqs are loaded if modes are specified
        print(f"\nNo frequencies loaded from {eigen_freq_file_s_main} or {eigen_freq_file_t_main} for the specified modes. Skipping calculation.")
    else:
        calculator_global = MoonPerturbationCalculatorGeneralized(
            all_spher_nl_tuples=all_spher_nl_tuples_main,
            all_tor_nl_tuples=all_tor_nl_tuples_main,
            full_omega_k2_map=full_omega_k2_map_from_file,
            model_file=model_file_path_main,
            eigenfunction_folder_spher=eigen_func_folder_s_main, 
            eigenfunction_folder_tor=eigen_func_folder_t_main,
            perturbations_input_dictionary=all_perturbations_main, 
            s_max=s_perturbation_degree_main
            # transform_matrix_threshold and delta_omega_ratio_threshold are removed
        )

        if calculator_global.num_k_modes > 0: 
            calculator_global.run_calculation()
            calculator_global.save_results(output_folder_base_main)
        else: 
            print(f"Skipping calculations and saving as no valid modes were found/processed for the specified N, L ranges globally.")


     # --- Independent Frequency Listing and Ratio Calculation ---
    print("\n--- Performing independent frequency listing and ratio calculation ---")
    if not full_omega_k2_map_from_file:
        print("No frequencies loaded (full_omega_k2_map_from_file is empty). Cannot list frequencies.")
    else:
        # Ensure output directory exists for this new file as well
        output_folder_base_main.mkdir(parents=True, exist_ok=True)
        sorted_freq_list_file = output_folder_base_main / "ALL_L_sorted_frequencies_with_ratios.txt"

        # Convert omega_k^2 to frequency in mHz and store with quantum numbers
        # k_tuple is (sigma_char, n, l)
        # omega_k2_map_from_file value is omega_k^2
        freq_data_list = []
        for k_tuple, omega_k2_val in full_omega_k2_map_from_file.items():
            sigma_char, n_val, l_val = k_tuple
            freq_hz = np.sqrt(omega_k2_val) / (2 * np.pi)
            freq_mhz = freq_hz * 1000.0
            freq_data_list.append({'sigma': sigma_char, 'n': n_val, 'l': l_val, 'freq_mhz': freq_mhz})

        # Sort by frequency
        sorted_freq_data = sorted(freq_data_list, key=lambda x: x['freq_mhz'])

        with open(sorted_freq_list_file, 'w') as f_sfl:
            f_sfl.write("# Sorted list of unperturbed frequencies for all S and T modes considered.\n")
            f_sfl.write("# Format: sigma_char, n_val, l_val, Freq(mHz), Ratio_Lower, Ratio_Upper\n")
            f_sfl.write("# Ratio_Lower = (Freq_current - Freq_previous) / Freq_current\n")
            f_sfl.write("# Ratio_Upper = (Freq_next - Freq_current) / Freq_current\n")
            f_sfl.write("# Ratios are 'nan' for the first and last frequency in the list.\n")

            num_freqs = len(sorted_freq_data)
            for i, data_item in enumerate(sorted_freq_data):
                s, n, l, f_mhz = data_item['sigma'], data_item['n'], data_item['l'], data_item['freq_mhz']
                
                ratio_lower_str = "nan"
                ratio_upper_str = "nan"

                if 0 < i < num_freqs -1 : # Only for items not at the ends
                    f_prev = sorted_freq_data[i-1]['freq_mhz']
                    f_next = sorted_freq_data[i+1]['freq_mhz']
                    
                    if abs(f_mhz) > 1e-9: # Avoid division by zero if frequency is tiny
                        ratio_lower = (f_mhz - f_prev) / f_mhz
                        ratio_upper = (f_next - f_mhz) / f_mhz
                        ratio_lower_str = f"{ratio_lower:.4f}" # 4 decimal places for ratio
                        ratio_upper_str = f"{ratio_upper:.4f}"
                    else:
                        ratio_lower_str = "div_zero"
                        ratio_upper_str = "div_zero"
                elif i == 0 and num_freqs > 1: # First item, if there's a next
                    f_next = sorted_freq_data[i+1]['freq_mhz']
                    if abs(f_mhz) > 1e-9:
                        ratio_upper = (f_next - f_mhz) / f_mhz
                        ratio_upper_str = f"{ratio_upper:.4f}"
                    else:
                        ratio_upper_str = "div_zero"
                elif i == num_freqs - 1 and num_freqs > 1: # Last item, if there's a previous
                    f_prev = sorted_freq_data[i-1]['freq_mhz']
                    if abs(f_mhz) > 1e-9:
                        ratio_lower = (f_mhz - f_prev) / f_mhz
                        ratio_lower_str = f"{ratio_lower:.4f}"
                    else:
                        ratio_lower_str = "div_zero"
                
                # If only one frequency, both ratios remain "nan"
                
                f_sfl.write(f"{s},{n},{l},{f_mhz:.6e},{ratio_lower_str},{ratio_upper_str}\n")
        print(f"Sorted frequencies with ratios saved to: {sorted_freq_list_file}")
    # --- End of Independent Frequency Listing ---

    print("\n\nFull Generalized Perturbation Calculation Example (Simplified) Finished.")
    print(f"Results, if any, saved in subfolder: {output_folder_base_main.resolve()}")
    print("********************************************************************************")
    print("*** This was an illustrative example run. Ensure paths and data are correct. ***")
    print("********************************************************************************")

Setting up Full Generalized Perturbation Calculation Example (Simplified)...
Attempting to load S-mode frequencies from: D:\Study\Research & Survey\Seismic GW detector\MyWork\Inverse-1D-pert\MultModel_Results1024\Reference_Model\res1.txt
  Successfully loaded relevant S-mode frequencies from D:\Study\Research & Survey\Seismic GW detector\MyWork\Inverse-1D-pert\MultModel_Results1024\Reference_Model\res1.txt
Attempting to load T-mode frequencies from: D:\Study\Research & Survey\Seismic GW detector\MyWork\LunarResponse\EigenCalculation\Reference_Model_toroidal\res1.txt
  No T-modes specified for loading frequencies.
Total 201 frequency entries compiled for specified (n,l) ranges.
Defined example perturbations up to s_max=1

--- Initializing Generalized Calculator for ALL specified L values ---
Loading background model from: D:\Study\Research & Survey\Seismic GW detector\MyWork\Inverse-1D-pert\MultModel_Results1024\Reference_Model\Reference_Model_8000_code.txt
Model loaded: 8000 radial poi

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 23 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 out of 201 | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Done  15 out of 201 | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done  26 out of 201 | elapsed:    1.5s
[Parallel(n_jobs=-2)]: Done  39 out of 201 | elapsed:    1.5s
[Parallel(n_jobs=-2)]: Done  52 out of 201 | elapsed:    1.6s
[Parallel(n_jobs=-2)]: Done  67 out of 201 | elapsed:    1.6s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.19615607042298705s.) Setting batch_size=2.
[Parallel(n_jobs=-2)]: Done  82 out of 201 | elapsed:    1.7s
[Parallel(n_jobs=-2)]: Done  99 out of 201 | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done 117 out of 201 | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.18705487251281738s.) Setting batch_size=4.
[Parallel(n_jobs=-2)]: Done 156 out of 201 | elapsed:    1.9s remaining:    0.5s
[Parallel(n_jobs=-2)]: Done 177 out of 201 | elapsed:    2.0s remaining:    0.2s
[Parallel(n


All 201 eigenfunctions processed and collected.
Calculating tilde V and T matrices (parallelized)...



[Parallel(n_jobs=-2)]: Using backend LokyBackend with 23 concurrent workers.
[Parallel(n_jobs=-2)]: Batch computation too fast (0.01952981948852539s.) Setting batch_size=2.
[Parallel(n_jobs=-2)]: Done     4 out of 40401 | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Done    15 out of 40401 | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Done    26 out of 40401 | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Done    39 out of 40401 | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.11908531188964844s.) Setting batch_size=4.
[Parallel(n_jobs=-2)]: Done    58 out of 40401 | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done    88 out of 40401 | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done   118 out of 40401 | elapsed:    0.2s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.17107892036437988s.) Setting batch_size=8.
[Parallel(n_jobs=-2)]: Done   166 out of 40401 | elapsed:    0.2s
[Parallel(n_jobs=-2)]: Done   234 out of 40401 | elapsed:    0.3s
[Parallel(n_jobs=-2)]: Done   310 out of 


Finished calculating tilde matrices.
Converting tilde matrices to real basis...
Finished converting to real basis matrices.
Calculating delta_omega_k_sq...
Finished delta_omega_k_sq.
All calculations finished for all specified L values.
Saving results to unified files in: TV_0S2-200S2_NoTor_matrix
delta_omega_km saved to TV_0S2-200S2_NoTor_matrix\ALL_L_delta_omega_km.txt
T_real_kk_mm saved to TV_0S2-200S2_NoTor_matrix\ALL_L_T_real_matrix.txt
V_real_kk_mm saved to TV_0S2-200S2_NoTor_matrix\ALL_L_V_real_matrix.txt

--- Performing independent frequency listing and ratio calculation ---
Sorted frequencies with ratios saved to: TV_0S2-200S2_NoTor_matrix\ALL_L_sorted_frequencies_with_ratios.txt


Full Generalized Perturbation Calculation Example (Simplified) Finished.
Results, if any, saved in subfolder: D:\Study\Research & Survey\Seismic GW detector\MyWork\Inverse-1D-pert\TV_0S2-200S2_NoTor_matrix
********************************************************************************
*** This was